Intra Exchange Trade

In [1]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Delta Neutral Real Trade)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo

# API Keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# Connect to APIs
bybit_spot = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget_spot = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bybit_futures = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'future'}
})

bitget_futures = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'swap'}
})


In [2]:
# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 500000
CHECK_INTERVAL = 2
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit_spot, 'Bitget': bitget_spot}
futures_apis = {'Bybit': bybit_futures, 'Bitget': bitget_futures}
futures_markets_cache = {}

# Helper Functions
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def wait_for_fill(exchange, order_id, symbol):
    print("🕒 Waiting for spot order to fill...")
    while True:
        order = exchange.fetch_order(order_id, symbol)
        if order['status'] == 'closed':
            print("✅ Spot order filled.")
            return order['filled']
        print("⏳ Still waiting for fill...")
        time.sleep(2)

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    print(f"🚀 Executing REAL delta-neutral trade on {exchange_name} - {symbol}")
    try:
        spot_exchange = exchanges[exchange_name]
        futures_exchange = futures_apis[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        # Calculate quantity
        amount = TRADE_USD_SIZE / spot_price

        # Spot BUY (limit order)
        print(f"📈 Placing SPOT LIMIT BUY: {amount:.6f} {base_asset} @ {spot_price:.4f}")
        order_spot = spot_exchange.create_order(
            symbol=symbol,
            type='limit',
            side='buy',
            amount=amount,
            price=spot_price
        )

        filled_amount = wait_for_fill(spot_exchange, order_spot['id'], symbol)
        if filled_amount == 0:
            return

        print(f"✅ Spot Order Filled: {filled_amount:.6f} {base_asset}")

        # Futures SELL (market order)
        print(f"📉 Placing FUTURES MARKET SELL: {filled_amount:.6f} {base_asset}")
        if exchange_name == 'Bitget':
            futures_exchange.set_position_mode(False, futures_symbol)
        futures_exchange.set_leverage(1, futures_symbol)
        order_futures = futures_exchange.create_order(
            symbol=futures_symbol,
            type='market',
            side='sell',
            amount=filled_amount
        )

        print(f"✅ Futures Order Placed: {order_futures['id']}")
        print(f"🏁 Delta-neutral trade completed for {symbol} on {exchange_name}")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol or ":USDT:USDT" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT") or f.startswith(f"{base}_USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(futures_apis[exchange_name], futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

✅ Found 222 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 246 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
✅ Cycle complete. Waiting for next check...
✅ Found 222 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...



KeyboardInterrupt

